In [3]:
#!pip install libs_qrem
#!pip install qiskit-ignis   

from qiskit import QuantumRegister, Aer, execute
from qiskit.ignis.mitigation.measurement import tensored_meas_cal
import numpy as np
import qiskit.circuit.library as qGate
from qiskit.extensions import UnitaryGate
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter
from qatg import QATG
from qatg import QATGFault
from libs_qrem import LeastNormFilter
from qiskit.tools.visualization import plot_histogram
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import standard_errors, ReadoutError

In [4]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import standard_errors, ReadoutError

def getNoiseModel_no_readouterror(self):
		# Depolarizing quantum errors
		oneQubitError = standard_errors.depolarizing_error(self.oneQubitErrorProb, 1)
		twoQubitError = standard_errors.depolarizing_error(self.twoQubitErrorProb, 2)
		qubitReadoutError = ReadoutError([self.zeroReadoutErrorProb, self.oneReadoutErrorProb])

		# Add errors to noise model
		noiseModel = NoiseModel()
		noiseModel.add_all_qubit_quantum_error(oneQubitError, self.basisGateSetString)
		noiseModel.add_all_qubit_quantum_error(twoQubitError, ['cx'])
		

		return noiseModel

In [5]:
def qatg_QREM(configuration):
    
    #first run the calibration circuit
    n = configuration.circuitSize
    qr = configuration.quantumRegister 
    mit_pattern = []
    for i in range(n):
        singlebit = [i]
        mit_pattern.append(singlebit)
    
    meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
    backend = Aer.get_backend('aer_simulator')
    
    no_readouterror_job = execute(meas_calibs, backend=backend, shots=200000,noise_model=getNoiseModel_no_readouterror(configuration))
    no_readouterror_cal_results = no_readouterror_job .result()
    print("no_readouterror")
    print(no_readouterror_cal_results.get_counts())
    
    noise_job = execute(meas_calibs, backend=backend, shots=200000, noise_model=configuration.getNoiseModel())
    noise_cal_results = noise_job.result()
    noisy_hist = noise_job.result().get_counts()
    print("noisy_hist")
    print(noisy_hist)
    
    meas_fitter = TensoredMeasFitter(noise_cal_results, mit_pattern=mit_pattern)
    print("calibration matrices")
    print(meas_fitter.cal_matrices)
    
    for i in range(n):
        print('Readout fidelity of Q'+str(i)+': %f'%meas_fitter.readout_fidelity(i))
        print('Q'+str(i)+' Calibration Matrix')
        meas_fitter.plot_calibration(i)
    
    
    
    meas_filter = LeastNormFilter(n, meas_fitter.cal_matrices)
    
    
    mitigated_hist = meas_filter.apply(noisy_hist)
    print("mitigated_hist")
    print(mitigated_hist)
    #count = mitigated_hist.get_counts()
    
    #display(plot_histogram([noisy_hist, count], legend=['raw', 'mitigated']))
    
    #run the fault_free circuit
    faultfree_noreadouterror = execute(configuration.faultfreeQCKT, backend=backend, shots=200000,noise_model=getNoiseModel_no_readouterror(configuration))
    faultfree_noreadouterror_results = faultfree_noreadouterror.result()
    # Results without mitigation
    
    no_readouterror = faultfree_noreadouterror_results.get_counts()
    print("no_readouterror")
    print(no_readouterror)
    
    
    
    faultfree_noisy_job = execute(configuration.faultfreeQCKT, backend=backend, shots=200000, noise_model=configuration.getNoiseModel())
    faultfree_noisy_results = faultfree_noisy_job.result()
    
    # Results without mitigation
    faultfree_raw_counts = faultfree_noisy_results.get_counts()

    # Get the filter object
    #meas_filter = meas_fitter.filter
    #print(meas_filter.cal_matrices)
    # Results with mitigation
    mitigated_results = meas_filter.apply(faultfree_noisy_results)
    mitigated_counts = mitigated_results.get_counts()
    display(plot_histogram([faultfree_raw_counts, mitigated_counts], legend=['raw', 'mitigated']))
    print("raw_counts")
    print(faultfree_raw_counts)
    print("mitigated_counts")
    print(mitigated_counts)
    